In [25]:
from imp import reload
import cProfile, time

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import cca_pytorch, cca, gp

import torch

In [26]:
T = 20
feats = 50
d = 5
K = gp.gen_kernel('squared_exp', temporal_scale=3., spatial_scale=.5)
Xt, _, _, _ = gp.embed_gp(100, feats, d, K, np.eye(feats), T, num_to_concat=1000)
CovXt = cca.calc_cross_cov_mats_from_data(Xt, 2 * T)

In [27]:
v = torch.tensor(np.random.randn(feats, d), requires_grad=True, device='cuda:0', dtype=torch.float64)
c = torch.tensor(CovXt, device='cuda:0', dtype=torch.float64)

In [28]:
optimizer = torch.optim.LBFGS([v], max_eval=15000, max_iter=15000)

In [29]:
def closure():
    optimizer.zero_grad()
    l = cca_pytorch.build_loss(c, d)(v)
    l.backward()
    return l

In [30]:
t = time.time()
for ii in range(1):
    print(cca_pytorch.build_loss(c, d)(v), v)
    optimizer.step(closure)
    print(cca_pytorch.build_loss(c, d)(v), v)
print(time.time() - t)

tensor(138314.5907, grad_fn=<AddBackward0>) tensor([[ 1.2248e+00, -1.6308e+00,  6.7192e-01,  5.6854e-01, -3.0518e-01],
        [-1.4563e+00,  1.2489e+00, -1.5771e-01, -1.9828e+00,  1.7819e+00],
        [-2.3920e-01, -1.3589e-01,  1.7186e+00,  2.5160e-03,  5.1443e-01],
        [ 5.9031e-01, -5.3350e-01,  4.7720e-01, -5.0363e-01, -2.5181e-01],
        [-2.4163e+00,  5.6849e-01, -6.4074e-01,  1.1621e+00,  4.0543e-01],
        [-1.3156e+00,  4.1506e-02,  8.5755e-01,  4.8158e-01,  4.5566e-01],
        [ 8.1716e-01,  1.9480e+00, -7.6442e-01, -5.3055e-01, -1.6628e-01],
        [ 7.0501e-01,  9.7047e-01, -6.8413e-01, -5.9497e-01,  4.9738e-01],
        [ 1.2380e+00,  8.3582e-01, -8.7702e-01,  4.7943e-01, -1.6125e+00],
        [ 4.5207e-01, -4.1735e-01, -5.6016e-01,  6.8190e-01, -1.2969e+00],
        [-7.4135e-01, -1.1842e+00,  1.5043e+00,  6.3809e-01,  9.8237e-01],
        [-1.2108e+00,  3.0519e-01, -5.6730e-01,  6.0957e-01,  2.1085e+00],
        [ 3.8116e-01,  9.3378e-01, -2.4054e-01, -6.2167e

In [31]:
t = time.time()
vp = cca.run_cca(CovXt, d, verbose=True)
print(time.time() - t)
print(cca.build_loss(CovXt, d)(vp.ravel()))

PI = 0.3494 bits, reg = 0.2781
PI = 0.3532 bits, reg = 0.0352
PI = 0.4554 bits, reg = 0.058
PI = 0.596 bits, reg = 0.1112
PI = 0.651 bits, reg = 0.1348
PI = 0.6818 bits, reg = 0.1087
PI = 0.7336 bits, reg = 0.1045
PI = 0.7381 bits, reg = 0.1004
PI = 0.7393 bits, reg = 0.0859
PI = 0.7453 bits, reg = 0.0789
PI = 0.752 bits, reg = 0.0607
PI = 0.7852 bits, reg = 0.0507
PI = 0.8497 bits, reg = 0.0567
PI = 0.8649 bits, reg = 0.0618
PI = 0.8795 bits, reg = 0.0589
PI = 0.9053 bits, reg = 0.0557
PI = 0.9468 bits, reg = 0.0537
PI = 1.0006 bits, reg = 0.0675
PI = 1.0465 bits, reg = 0.0618
PI = 1.0965 bits, reg = 0.0723
PI = 1.1307 bits, reg = 0.0854
PI = 1.1409 bits, reg = 0.0669
PI = 1.155 bits, reg = 0.0388
PI = 1.2032 bits, reg = 0.0235
PI = 1.2396 bits, reg = 0.0181
PI = 1.2474 bits, reg = 0.0203
PI = 1.2566 bits, reg = 0.0188
PI = 1.2764 bits, reg = 0.0202
PI = 1.3098 bits, reg = 0.0289
PI = 1.3435 bits, reg = 0.0231
PI = 1.4101 bits, reg = 0.0616
PI = 1.4279 bits, reg = 0.0399
PI = 1.4573 b

PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, reg = 0.0
PI = 2.0998 bits, re